# Importaciones y funciones necesarias

In [2]:
#importamos la librerias necesarias
import pandas as pd
import ast

In [3]:
#creamos una funcion para buscar el indice name 
def buscar_nombre(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    # en caso de haber mas de un diccionario con la clave name devuelve todos los valores concatenados
    return ', '.join([i['name'] for i in lista])

#creamos otra funcion con la misma logica  que buscar nombre pero modificada para funcionar con diccionarios
def buscar_nombre_diccionario(dicc):
    #en este caso esto convierte los objetos a diccionarios
    dicc = ast.literal_eval(dicc)
    return dicc['name']

#creamos una funcion para buscar el indice name si el job es igual a director
def buscar_nombre_director(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    for i in lista:
            if i['job'] == 'Director':
                return i['name']
            else: continue


# ETL de movies_dataset.csv

In [7]:
#leemos el csv
movies = pd.read_csv('data/movies_dataset.csv')
#comprobamos que el dataframe este cargado correctamente
movies

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 1745


In [22]:
#comprobamos el tamaño del dataframe
print(movies.shape)
#removemos los valores duplicados
movies.drop_duplicates(inplace=True)
print(movies.shape)
#removemos las columnas que no se utilizaran y/o estan duplicadas
movies = movies.drop(columns=['video', 'adult', 'imdb_id', 'original_title', 'poster_path', 'homepage', 'overview','tagline'])
print(movies.shape)


(45466, 24)
(45453, 24)
(45453, 16)


In [23]:
#remplazamos los nulos por 0
movies['revenue'].replace(to_replace=None,regex=0 ,inplace=True)
movies['budget'].replace(to_replace=None,regex=0 ,inplace=True)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_1888\163287723.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies['revenue'].replace(to_replace=None,regex=0 ,inplace=True)
C:\Users\Miguel\AppData\Local\Temp\ipykernel_1888\163287723.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [24]:
#removemos las filas con valores faltantes en la columna release_date
movies = movies.dropna(subset=['release_date'])
#eliminamos las filas que no sigan el formato AAAA-MM-DD en la columna release_date
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')
movies = movies.dropna(subset=['release_date'])

In [25]:
#creamos la columna year 
movies['released_year'] = movies['release_date'].dt.year

In [26]:
#cambiamos el tipo de las columnas budget y revenue a int para poder hacer operaciones
movies['budget'] = movies['budget'].astype(int)
movies['revenue'] = movies['revenue'].astype(int)
#creamos la columna return que es la columna revenue entre la columna budget
    #en caso de que uno de los numeros sea 0 devolvera 0
movies['return'] = movies.apply(lambda fila: fila['revenue'] / fila['budget'] if fila['budget'] != 0 else 0, axis=1)


In [27]:
#aplicamos la funcion buscar_nombre a las columnas para desanidar los datos 
movies['genres'] = movies['genres'].fillna('[]').apply(buscar_nombre)
movies['spoken_languages'] = movies['spoken_languages'].fillna('[]').apply(buscar_nombre)
movies['production_companies'] = movies['production_companies'].fillna('[]').apply(buscar_nombre)
movies['production_countries'] = movies['production_countries'].fillna('[]').apply(buscar_nombre)

In [28]:
#en caso de no haber datos en belongs_to_collection la funcion fillna crea un diccionario con la clave name y el valor "NA"
    #de no hacerlo daria un key error al ejecutar la funcion buscar_nombre_diccionario
movies['belongs_to_collection'] = movies['belongs_to_collection'].fillna('{"name":"NA"}').apply(buscar_nombre_diccionario)

In [29]:
movies

,belongs_to_collection,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,released_year,return
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033,81.0,English,Released,Toy Story,7.7,5415.0,1995,12.451801
1,NA,65000000,"Adventure, Fantasy, Family",8844,en,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249,104.0,"English, Français",Released,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,0,"Romance, Comedy",15602,en,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0,101.0,English,Released,Grumpier Old Men,6.5,92.0,1995,0.000000
3,NA,16000000,"Comedy, Drama, Romance",31357,en,3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156,127.0,English,Released,Waiting to Exhale,6.1,34.0,1995,5.090760
4,Father of the Bride Collection,0,Comedy,11862,en,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911,106.0,English,Released,Father of the Bride Part II,5.7,173.0,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NA,0,"Drama, Action, Romance",30840,en,5.683753,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",1991-05-13,0,104.0,English,Released,Robin Hood,5.7,26.0,1991,0.000000
45462,NA,0,Drama,111109,tl,0.178241,Sine Olivia,Philippines,2011-11-17,0,360.0,,Released,Century of Birthing,9.0,3.0,2011,0.000000
45463,NA,0,"Action, Drama, Thriller",67758,en,0.903007,American World Pictures,United States of America,2003-08-01,0,90.0,English,Released,Betrayal,3.8,6.0,2003,0.000000
45464,NA,0,,227506,en,0.003503,Yermoliev,Russia,1917-10-21,0,87.0,,Released,Satan Triumphant,0.0,0.0,1917,0.000000


# etl de credits.csv

In [30]:
#leemos el csv
credits = pd.read_csv('credits.csv')
#comprobamos que el dataset se haya cargado correctamente
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [31]:
#comprobamos el tamaño del dataframe
print(credits.shape)
# eliminamos los duplicados y volvemos a comprobar el tamaño del dataframe
credits.drop_duplicates(inplace=True)
print(credits.shape)

(45476, 3)
(45439, 3)


In [32]:
# obtenemos los nombres de los actores
credits['cast'] = credits['cast'].fillna('[]').apply(buscar_nombre)

In [33]:
#obtenemos los nombres de los directores
credits['director'] = credits['crew'].apply(buscar_nombre_director)

In [34]:
#eliminamos la columna crew para reducir significativamente el peso del dataframe
credits = credits.drop(columns='crew')

In [35]:
credits

,cast,id,director
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",862,John Lasseter
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",8844,Joe Johnston
2,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",15602,Howard Deutch
3,"Whitney Houston, Angela Bassett, Loretta Devin...",31357,Forest Whitaker
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",11862,Charles Shyer
...,...,...,...
45471,"Leila Hatami, Kourosh Tahami, Elham Korda",439050,Hamid Nematollah
45472,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",111109,Lav Diaz
45473,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",67758,Mark L. Lester
45474,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",227506,Yakov Protazanov


# concatenamos y exportamos como csv

In [36]:
movies = pd.concat([movies, credits], axis=1, join='inner')

movies_limpio_path = 'data/movies_limpio.csv'

movies.to_csv(movies_limpio_path, sep=',', index=False, encoding='utf-8')
